## Orthogroup-lens correlation and GO enrichment analysis

In this notebook, we perform a GO enrichment analysis on orthogroups that are highly correlated (positively and negatively) with the *stress* and *tissue* lenses we used to construct mapper graphs. the mapper algorithm to the gene expression data.

### import packages
First thing, import all the necessary packages and methods. Most of these are commonly used python packages, that can be easily installed. The only uncommon package is `goatools` (for: *Gene Ontology Analysis tools*), which can be pip installed. For more information and documentation, check out [this link](https://pypi.org/project/goatools/). We will use the *multipletests* method from the *statsmodels* package to correct for multiple testing when performing statistical tests.

We also need to import custom functions defined in  `helper_functions.py`. As the name suggests, one contains several utility functions required to perform tasks such as loading and scaling input data. The other contains the function to compute the *lens* (more on that further down in the notebook).

In [ ]:
# imports

import os
import numpy as np
import pandas as pd
import pickle
from scipy import stats
from statsmodels.stats.multitest import multipletests
from helper_functions import loaddata

from goatools.utils import read_geneset
from goatools.obo_parser import GODag
from goatools.anno.idtogos_reader import IdToGosReader
from goatools.go_enrichment import GOEnrichmentStudy
from goatools.base import download_go_basic_obo

### set paths, specify file names
Next thing we want to do is set the paths to the directories containing `code`, `data`, and `results`. We also specify input files. The data to be analyzed is stored in two csv files. `clean_metadata.csv` contains the metadata, `clean_RNAseq_OutlierRemoved.csv` contains the gene expression data. 

The metadata file contains one sample per row, identified by its *SRA*. There are 3172 samples, and for each sample we have 7 descriptive attributes. We are particularly interested in three of them: namely, its plant *family*, *tissue* type and *stress* type. There are 16 different plant families, 8 tissue types and 10 stress types (including *healthy*) represented in the data. The RNAseq file, as the name suggests contains the gene expression for each sample, across a 6335 orthogroups. 

In [ ]:
# load data
projdir = "../.."
datadir = projdir + "/data"
resdir = projdir + "/results/goea"

factorfile = datadir + "/clean_metadata.csv"
rnafile = datadir + "/raw_RNAseq_OutlierRemoved.csv"

### Set factor and level for used for the lens

Here, we set the factor, and the corresponding factor level that was used to construct the lens function. In the manuscript we presented the results using the following three combinations:
- filterfactor: *stress*, filterlevel: *healthy*
- filterfactor: *tissue*, filterlevel: *root*
- filterfactor: *tissue*, filterlevel: *leaf*

These variables are also used to construct the names of files and directories in which the results will be saved.

In [ ]:
# Cover parameter ranges for specified lenses
factors = ["stress", "tissue", "family"]
filterfactor, filterlevel = ("tissue", "leaf")
if filterfactor == "stress":
    cubes, overlap = (110, 75)
elif filterfactor == "tissue":
    if filterlevel == "root":
        cubes, overlap = (110, 90)
    elif filterlevel == "leaf":
        cubes, overlap = (140, 90)
    else:
        print("filterlevel must be root or leaf")
else:
    print("filterfactor must be stress or tissue")

### Load the data and the mapper graph

The input data (metadata and gene expression) is stored in two separate files, but samples can be matched using the *SRA*. The function `loaddata` does exactly that. It loads both files into pandas dataframes and merges them using the *SRA* as the key. We perform an inner join, to keep only those *SRAs* for which we have metadata as well as gene expression. We also load the saved mapper graph which contains the lens function values used to construct it.

In [ ]:
# load data
df, orthos = loaddata(factorfile, rnafile, factors)

# load saved mapper graph
mapf = f"{filterfactor}_{filterlevel}_mapper_cubes_{cubes}_overlap_{overlap}"
mapf = datadir + "/saved_mapper_graphs/" + mapf + ".pkl"
with open(mapf, "rb") as fp:
    graph = pickle.load(fp)


### Orthogroup-lens correlation

Now, for each of the 6335 orthogroups, we will compute the Pearson correlation between the orthogroup expression and the lens function values across all samples. Once we have all the correlation coefficients and the corresponding p-values, we will adjust them for multiple testing and put all that data into a pandas dataframe. Uncomment the last two lines of the following cell if you wish to save the dataframe.

In [ ]:
lens = np.asarray(graph["lens"]).squeeze()
corrs = []
pvals = []

# Compute orthogroup-lens correlation with p-value for each orthogroup
for og in orthos:
    r, pv = stats.pearsonr(lens, np.asarray(df[og]).squeeze())
    corrs.append(r)
    pvals.append(pv)

# Compute p-values adjusted for multiple testing
reject, p_adjust, _, _ = multipletests(pvals, alpha=0.001, method='fdr_bh')

# Create a dataframe
corrdf = pd.DataFrame({"Orthogroup": orthos, "correlation": corrs,
                       "pvalues": pvals, "adjusted_pvalues": p_adjust})

# # Uncomment to save the dataframe
# fname = f"/goea/{filterfactor}_{filterlevel}_lens_orthogroup_correlations.csv"
# corrdf.to_csv(resdir + fname, index=False)

### Left and Right tails of the correlation value distribution

We have 6335 correlation coefficients. For GO enrichment analysis, we select the 2.5% of the most positive and 2.5% of the most negative coefficient values. This is same as computing 5% of the values that fall in the left and right tails of the distribution of all 6335 coefficients.

In [ ]:
#Filter correlation dataframe to get extreme values (left and right tails)
left = 0.025
right = 0.975
tails = corrdf["correlation"].quantile(q=[left, right])
corrdf_left = corrdf.loc[corrdf["correlation"] < tails[left]]
corrdf_right = corrdf.loc[corrdf["correlation"] > tails[right]]

### Load orthogroup-GO terms association file

This file contains the association between orthogroups, the corresponding TAIR loci and the GO terms associated with those TAIR loci.

In [ ]:
# Load the file that maps orthogroups to GO terms using TAIR loci
fname = datadir + "/goea/Orthogroup_GO_terms.tsv"
godf = pd.read_csv(fname, sep="\t")
column_names = ["Orthogroup", "TAIRLocus", "GO term", "GO ID", "GO Slim(s)"]
godf.drop_duplicates(subset=column_names, keep="first", inplace=True)

### Prepare input files for GO enrichment analysis

To perform GO enrichment analysis using `goatools`, we need to create input files in the specified formats. First, we need a text file listing all the TAIR loci in the *population* - that is, a text file listing all unique TAIR loci associated with the orthogroups in our data. Second, we need a text file containing the associations between TAIR loci and the GO IDs.

In [ ]:
# List all unique TAIR loci associated with orthogroups and write to a file
tair_file = datadir + "/goea/all_tairs.txt"
with open(tair_file, "w") as fp:
    fp.write("\n".join(godf["TAIRLocus"].unique()))

# Create TAIR Locus - GO ID associations and write to file
assoc_file = datadir + "/goea/go_association.txt"
tair_grouped = godf.groupby("TAIRLocus")["GO ID"].aggregate(set)
with open(assoc_file, "w") as fp:
    for t in tair_grouped.index:
        fp.write(t + "\t" + ";".join(list(tair_grouped[t])) + "\n")

We will perform separate GO enrichment analyses for the set of orthogroups most negatively correlated with the lense (prefix: *left_* for left tail), and the set of orthogroups most positively correlated with the lense (prefix: *right_*, for right tail). For GO enrichment analysis, we need to identify the lists of unique TAIR loci associated with those sets of orthogroups and write them to text files.

In [ ]:
# List TAIR loci associated with orthogroups in the left tail and write to file
left_file = f"/goea/{filterfactor}_{filterlevel}_lens_left_tail_tairs.txt"
left_file = datadir + left_file
left_godf = godf[godf["Orthogroup"].isin(corrdf_left["Orthogroup"])]
with open(left_file, "w") as fp:
    fp.write("\n".join(left_godf["TAIRLocus"].unique()))

# List TAIR loci associated with orthogroups in the right tail and write to file
right_file = f"/goea/{filterfactor}_{filterlevel}_lens_right_tail_tairs.txt"
right_file = datadir + right_file
right_godf = godf[godf["Orthogroup"].isin(corrdf_right["Orthogroup"])]
with open(right_file, "w") as fp:
    fp.write("\n".join(right_godf["TAIRLocus"].unique()))

### Go enrichment analysis

Once the input files are ready, we will perform the go enrichment analysis (separately for the positively and negatively correlated orthogroups). In addition to the files containing all TAIR loci, test subset of TAIR loci, and the GO associations, we also need the basic gene ontology annotation from `go-basic.obo`. This file has GO annotations filtereed in such a way that they are acyclic and can be propagated up the graph. If this file is not available in the specified location, goatools can download it for you. Once we set all the required variables and parameters, GO enrichment analysis runs pretty quickly.

In [ ]:
# GO enrichment for a subset of TAIR Loci (left/right tail) against all loci
fin_pop   = tair_file
fin_anno  = assoc_file

# Specify which tail: "left" or "right"
tail = "left"
if tail == "left":
    fin_study = left_file
else:
    fin_study = right_file

# Load the go-basic.obo file if it exists, else download it.
fin_obo   = datadir + "/goea/go-basic.obo"
if not os.path.isfile(fin_obo):
    fin_obo = download_go_basic_obo()

# Populate necessary variables to run GO Enrichment
study_ids = read_geneset(fin_study)
population_ids = read_geneset(fin_pop)
godag = GODag(fin_obo)
annoobj = IdToGosReader(fin_anno, godag=godag)
id2gos = annoobj.get_id2gos()

# GO Enrichment
goeaobj = GOEnrichmentStudy(
    population_ids,
    annoobj.get_id2gos(),
    godag,
    methods=['bonferroni', 'fdr_bh'],
    pvalcalc='fisher_scipy_stats')

results = goeaobj.run_study_nts(study_ids)

### Writing results to a file

The results of GO enrichment analysis are returned in form of an inconvenient object. To make it easier, we will convert and save them into a simple `.csv` file. We only want to see statistically significant results, i.e., when p-value is less than 0.05.

In [ ]:
# Collect GO Enrichment results, format and write to csv file.
goea_out = f"/goea/{filterfactor}_{filterlevel}_lens_{tail}_tail_goea_results"
goea_out = datadir + goea_out + ".csv"

# Column headers
hdrs = ["Namespace", "GO ID", "Enriched/Purified", "GO Terms", "pval_uncorr", "Benjamimi/Hochberg", "Bonferroni", "Study Ratio", "Population Ratio"]

# Row values pattern
pat = "{NS}\t{GO}\t{e}\t{GOTERM}\t{PVAL:8.2e}\t\
       {BH:8.2e}\t{BONF:8.2e}\t{RS:>12}\t{RP:>12}\n"

# Write to file, one row at a time: iff adjusted p-value < 0.05
with open(goea_out, "w") as fp:
    fp.write("\t".join(hdrs) + "\n")
    for ntd in sorted(results, key=lambda nt: [nt.p_uncorrected, nt.GO]):
        if ntd.p_fdr_bh < 0.05:
            ogs = godf[godf["GO ID"] == ntd.GO]["Orthogroup"].tolist()
            fp.write(pat.format(
                    NS=ntd.NS,
                    GO=ntd.GO,
                    e=ntd.enrichment,
                    GOTERM=ntd.goterm.name,
                    RS='{}/{}'.format(*ntd.ratio_in_study),
                    RP='{}/{}'.format(*ntd.ratio_in_pop),
                    PVAL=ntd.p_uncorrected,
                    BONF=ntd.p_bonferroni,
                    BH=ntd.p_fdr_bh))

print(f"Wrote results to {goea_out}")